# Machine Learning Prediction Models

## Introdução

Como parte final do trabalho, agora é realizada a previsão das tendências do preço utilizando algoritmos de aprendizagem de máquina aliado ao melhor modelo de previsão utilizando indicadores técnicos: SMA (Simple Moving Average).

In [1]:
matplotlib inline

In [2]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
wowTokenAppendedData = []
regions = ['NA', 'EU', 'CN', 'KR', 'TW']

for entry in os.scandir('./input/wowtoken'):
    if entry.is_file():
        wowTokenEntry = pd.read_csv(entry.path)
        wowTokenEntry['region'] = (os.path.splitext(entry.name)[0])
        wowTokenEntry['date'] = pd.to_datetime(wowTokenEntry['date'])
        wowTokenAppendedData.append(wowTokenEntry)
        
data = pd.concat(wowTokenAppendedData)

In [4]:
data.pivot(columns='region', values='price').describe()

region,CN,EU,KR,NA,TW
count,6835.000000,7744.000000,7642.000000,7886.000000,7468.000000
mean,242416.955377,164289.345687,271204.557969,95573.815876,260885.958356
std,168907.809419,101425.233611,98936.313849,64075.637750,104487.473796
min,48604.000000,30352.000000,121305.000000,18296.000000,114619.000000
25%,83650.000000,68769.250000,179645.500000,37004.250000,174211.000000
50%,215554.000000,171327.000000,285099.500000,89460.500000,213519.000000
75%,400670.000000,256468.750000,350899.250000,163553.250000,335554.500000
max,586090.000000,401827.000000,595930.000000,238572.000000,501220.000000


In [5]:
dataNA = data.loc[data['region'] == 'NA'].drop(['date', 'region'], axis=1)
dataCN = data.loc[data['region'] == 'CN'].drop(['date', 'region'], axis=1)
dataEU = data.loc[data['region'] == 'EU'].drop(['date', 'region'], axis=1)
dataKR = data.loc[data['region'] == 'KR'].drop(['date', 'region'], axis=1)
dataTW = data.loc[data['region'] == 'TW'].drop(['date', 'region'], axis=1)

In [6]:
def calculate_short_SMA(prices, period):
    if len(prices) < period:
        return 0
    
    return np.mean(prices[-10:])
    
def calculate_long_SMA(prices, period):
    if len(prices) < period:
        return 0
    
    return np.mean(prices[-50:])

def calculate_SMAs(data):
    prices = data['price'].values
    
    shortSMAs = []
    longSMAs = []
    pricesSeen = []
    for price in prices:
        pricesSeen.append(price)
        
        shortSMAs.append(calculate_short_SMA(pricesSeen, 10))
        longSMAs.append(calculate_long_SMA(pricesSeen, 50))
        
    data['short_sma'] = shortSMAs
    data['long_sma'] = longSMAs
    
    return data

In [7]:
dataNA = calculate_SMAs(dataNA)
dataCN = calculate_SMAs(dataCN)
dataEU = calculate_SMAs(dataEU)
dataKR = calculate_SMAs(dataKR)
dataTW = calculate_SMAs(dataTW)

In [8]:
def make_prediction(data, index):
    if index == 0:
        return 0
    
    else:
        shortSMA = data.loc[index, 'short_sma']
        lastShortSMA = data.loc[index-1, 'short_sma']
        longSMA = data.loc[index, 'long_sma']
        lastLongSMA = data.loc[index-1, 'long_sma']
        
        if shortSMA > lastShortSMA and longSMA > lastLongSMA:
            # Both SMAs are increasing, so the tendency is to rise
            return 1

        elif shortSMA <= lastShortSMA and longSMA <= lastLongSMA:
            # Both SMAs are decreasing, so the tendency is to fall
            return 0

        elif lastShortSMA <= longSMA and shortSMA > longSMA:
            # The short SMA crossed the long SMA by increasing itself, so in this case,
            # we hope that the short SMA goes back to the long SMA, so the tendency is to fall
            return 0

        elif lastShortSMA > longSMA and shortSMA <= longSMA:
            # The short SMA crossed the long SMA by decreasing itself, so in this case,
            # we hope that the short SMA goes back to the long SMA, so the tendency is to rise
            return 1
        
        else:
            return 0

In [9]:
preds = []
for index in range(len(dataNA)):
    preds.append(make_prediction(dataNA, index))
    
dataNA['simple_prediction'] = preds

previousPrices = [0]
previousPrices.extend(dataNA['price'][:-1])
dataNA['previous_price'] = previousPrices

riseOrDecrease = [0]
for index in range(len(dataNA['price'])):
    if index > 0:
        riseOrDecrease.append(1 if dataNA['price'][index] > dataNA['price'][index-1] else 0)
dataNA['rise_or_decrease'] = riseOrDecrease

dataNA = dataNA.loc[dataNA['long_sma'] != 0.0]
dataNA = dataNA.reset_index()

In [10]:
preds = []
for index in range(len(dataCN)):
    preds.append(make_prediction(dataCN, index))
    
dataCN['simple_prediction'] = preds

previousPrices = [0]
previousPrices.extend(dataCN['price'][:-1])
dataCN['previous_price'] = previousPrices

riseOrDecrease = [0]
for index in range(len(dataCN['price'])):
    if index > 0:
        riseOrDecrease.append(1 if dataCN['price'][index] > dataCN['price'][index-1] else 0)
dataCN['rise_or_decrease'] = riseOrDecrease

dataCN = dataCN.loc[dataCN['long_sma'] != 0.0]
dataCN = dataCN.reset_index()

In [11]:
preds = []
for index in range(len(dataEU)):
    preds.append(make_prediction(dataEU, index))
    
dataEU['simple_prediction'] = preds

previousPrices = [0]
previousPrices.extend(dataEU['price'][:-1])
dataEU['previous_price'] = previousPrices

riseOrDecrease = [0]
for index in range(len(dataEU['price'])):
    if index > 0:
        riseOrDecrease.append(1 if dataEU['price'][index] > dataEU['price'][index-1] else 0)
dataEU['rise_or_decrease'] = riseOrDecrease

dataEU = dataEU.loc[dataEU['long_sma'] != 0.0]
dataEU = dataEU.reset_index()

In [12]:
preds = []
for index in range(len(dataKR)):
    preds.append(make_prediction(dataKR, index))
    
dataKR['simple_prediction'] = preds

previousPrices = [0]
previousPrices.extend(dataKR['price'][:-1])
dataKR['previous_price'] = previousPrices

riseOrDecrease = [0]
for index in range(len(dataKR['price'])):
    if index > 0:
        riseOrDecrease.append(1 if dataKR['price'][index] > dataKR['price'][index-1] else 0)
dataKR['rise_or_decrease'] = riseOrDecrease

dataKR = dataKR.loc[dataKR['long_sma'] != 0.0]
dataKR = dataKR.reset_index()

In [13]:
preds = []
for index in range(len(dataTW)):
    preds.append(make_prediction(dataTW, index))
    
dataTW['simple_prediction'] = preds

previousPrices = [0]
previousPrices.extend(dataTW['price'][:-1])
dataTW['previous_price'] = previousPrices

riseOrDecrease = [0]
for index in range(len(dataTW['price'])):
    if index > 0:
        riseOrDecrease.append(1 if dataTW['price'][index] > dataTW['price'][index-1] else 0)
dataTW['rise_or_decrease'] = riseOrDecrease

dataTW = dataTW.loc[dataTW['long_sma'] != 0.0]
dataTW = dataTW.reset_index()

In [14]:
dataNA.head()

,index,price,short_sma,long_sma,simple_prediction,previous_price,rise_or_decrease
0,49,22279,22510.2,23305.68,1,21430,1
1,50,23184,22707.2,23169.36,0,22279,1
2,51,24067,22935.7,23032.58,0,23184,1
3,52,24407,23109.8,22950.42,0,24067,1
4,53,23699,23121.0,22923.92,0,24407,0


In [15]:
dataCN.head()

,index,price,short_sma,long_sma,simple_prediction,previous_price,rise_or_decrease
0,49,60523,64428.9,62634.98,0,68281,0
1,50,55157,62597.2,62766.04,1,60523,0
2,51,62050,61775.4,62784.96,0,55157,1
3,52,69802,62527.2,62857.72,1,62050,1
4,53,75015,64507.9,62896.94,1,69802,1


In [16]:
dataEU.head()

,index,price,short_sma,long_sma,simple_prediction,previous_price,rise_or_decrease
0,49,37917,40759.2,38370.60,0,42778,0
1,50,33610,39749.2,38342.80,0,37917,0
2,51,33308,38889.2,38188.14,0,33610,0
3,52,37293,38892.5,38046.16,0,33308,1
4,53,41664,39634.7,38054.14,1,37293,1


In [17]:
dataKR.head()

,index,price,short_sma,long_sma,simple_prediction,previous_price,rise_or_decrease
0,49,157885,157025.2,154153.28,1,164368,0
1,50,154136,156751.8,154765.54,0,157885,0
2,51,156231,156468.4,155116.70,0,154136,1
3,52,159090,157041.0,155185.58,1,156231,1
4,53,160938,159214.7,154915.30,0,159090,1


In [18]:
dataTW.head()

,index,price,short_sma,long_sma,simple_prediction,previous_price,rise_or_decrease
0,49,280574,267633.5,266395.04,1,286498,0
1,50,249258,263891.6,266380.20,0,280574,0
2,51,230263,255792.2,265481.86,0,249258,0
3,52,235784,250359.4,264022.42,0,230263,1
4,53,244735,249117.7,262064.44,0,235784,1


In [19]:
from keras.layers import (LSTM, Activation, Bidirectional, Dense, Embedding,
                          InputLayer, TimeDistributed)
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

class NeuralNetwork:
    def __init__(self, timesteps, data_dim):
        self.model = Sequential()
        self.model.add(LSTM(30, input_shape=(timesteps, data_dim)))
        self.model.add(Dense(2, activation='softmax'))

        self.model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

        self.model.summary()

    def train(self, x_train, y_train):
        self.model.fit(x_train, y_train, epochs=10, validation_split=0.2, verbose=2)

Using TensorFlow backend.


In [20]:
y_target = to_categorical(dataNA['rise_or_decrease'])

x_train, x_test, y_train, y_test = train_test_split(dataNA[['short_sma', 'long_sma', 'previous_price', 'simple_prediction']].values, y_target, test_size=0.33)

x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

nn = NeuralNetwork(1, x_train.shape[2])
nn.train(x_train, y_train)

lstm_accNA = nn.model.evaluate(x_test, y_test)[1]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30)                4200      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 62        
Total params: 4,262
Trainable params: 4,262
Non-trainable params: 0
_________________________________________________________________
Train on 4200 samples, validate on 1050 samples
Epoch 1/10
 - 2s - loss: 0.7564 - acc: 0.5081 - val_loss: 0.6940 - val_acc: 0.5057
Epoch 2/10
 - 0s - loss: 0.6939 - acc: 0.5017 - val_loss: 0.6934 - val_acc: 0.5057
Epoch 3/10
 - 0s - loss: 0.6937 - acc: 0.4962 - val_loss: 0.6934 - val_acc: 0.5057
Epoch 4/10
 - 1s - loss: 0.6936 - acc: 0.5019 - val_loss: 0.6934 - val_acc: 0.5057
Epoch 5/10
 - 0s - loss: 0.6933 - acc: 0.4933 - val_loss: 0.6935 - val_acc: 0.5057
Epoch 6/10
 - 0s - loss: 0.6931 - acc: 0.4945 - val_loss: 0.6938 - val_ac

In [21]:
y_target = to_categorical(dataCN['rise_or_decrease'])

x_train, x_test, y_train, y_test = train_test_split(dataCN[['short_sma', 'long_sma', 'previous_price', 'simple_prediction']].values, y_target, test_size=0.33)

x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

nn = NeuralNetwork(1, x_train.shape[2])
nn.train(x_train, y_train)

lstm_accCN = nn.model.evaluate(x_test, y_test)[1]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 30)                4200      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 62        
Total params: 4,262
Trainable params: 4,262
Non-trainable params: 0
_________________________________________________________________
Train on 3636 samples, validate on 910 samples
Epoch 1/10
 - 1s - loss: 0.8681 - acc: 0.5176 - val_loss: 0.7227 - val_acc: 0.5110
Epoch 2/10
 - 0s - loss: 0.7022 - acc: 0.5182 - val_loss: 0.6960 - val_acc: 0.5110
Epoch 3/10
 - 0s - loss: 0.6955 - acc: 0.5182 - val_loss: 0.6945 - val_acc: 0.5110
Epoch 4/10
 - 0s - loss: 0.6953 - acc: 0.5182 - val_loss: 0.6946 - val_acc: 0.5110
Epoch 5/10
 - 0s - loss: 0.6953 - acc: 0.5061 - val_loss: 0.6947 - val_acc: 0.5110
Epoch 6/10
 - 0s - loss: 0.6951 - acc: 0.5182 - val_loss: 0.6942 - val_acc

In [22]:
y_target = to_categorical(dataEU['rise_or_decrease'])

x_train, x_test, y_train, y_test = train_test_split(dataEU[['short_sma', 'long_sma', 'previous_price', 'simple_prediction']].values, y_target, test_size=0.33)

x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

nn = NeuralNetwork(1, x_train.shape[2])
nn.train(x_train, y_train)

lstm_accEU = nn.model.evaluate(x_test, y_test)[1]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30)                4200      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 62        
Total params: 4,262
Trainable params: 4,262
Non-trainable params: 0
_________________________________________________________________
Train on 4124 samples, validate on 1031 samples
Epoch 1/10
 - 1s - loss: 0.7002 - acc: 0.5160 - val_loss: 0.6937 - val_acc: 0.5063
Epoch 2/10
 - 0s - loss: 0.6935 - acc: 0.5155 - val_loss: 0.6935 - val_acc: 0.5092
Epoch 3/10
 - 0s - loss: 0.6934 - acc: 0.5145 - val_loss: 0.6942 - val_acc: 0.5063
Epoch 4/10
 - 0s - loss: 0.6936 - acc: 0.5155 - val_loss: 0.6936 - val_acc: 0.5092
Epoch 5/10
 - 0s - loss: 0.6937 - acc: 0.5153 - val_loss: 0.6935 - val_acc: 0.5092
Epoch 6/10
 - 0s - loss: 0.6932 - acc: 0.5155 - val_loss: 0.6940 - val_ac

In [23]:
y_target = to_categorical(dataKR['rise_or_decrease'])

x_train, x_test, y_train, y_test = train_test_split(dataKR[['short_sma', 'long_sma', 'previous_price', 'simple_prediction']].values, y_target, test_size=0.33)

x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

nn = NeuralNetwork(1, x_train.shape[2])
nn.train(x_train, y_train)

lstm_accKR = nn.model.evaluate(x_test, y_test)[1]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 30)                4200      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 62        
Total params: 4,262
Trainable params: 4,262
Non-trainable params: 0
_________________________________________________________________
Train on 4069 samples, validate on 1018 samples
Epoch 1/10
 - 1s - loss: 0.7052 - acc: 0.5119 - val_loss: 0.6913 - val_acc: 0.5059
Epoch 2/10
 - 0s - loss: 0.6893 - acc: 0.5306 - val_loss: 0.6886 - val_acc: 0.5334
Epoch 3/10
 - 0s - loss: 0.6896 - acc: 0.5058 - val_loss: 0.6890 - val_acc: 0.5059
Epoch 4/10
 - 0s - loss: 0.6888 - acc: 0.5200 - val_loss: 0.6897 - val_acc: 0.5059
Epoch 5/10
 - 0s - loss: 0.6886 - acc: 0.5284 - val_loss: 0.6878 - val_acc: 0.5334
Epoch 6/10
 - 0s - loss: 0.6883 - acc: 0.5259 - val_loss: 0.6893 - val_ac

In [24]:
y_target = to_categorical(dataTW['rise_or_decrease'])

x_train, x_test, y_train, y_test = train_test_split(dataTW[['short_sma', 'long_sma', 'previous_price', 'simple_prediction']].values, y_target, test_size=0.33)

x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

nn = NeuralNetwork(1, x_train.shape[2])
nn.train(x_train, y_train)

lstm_accTW = nn.model.evaluate(x_test, y_test)[1]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 30)                4200      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 62        
Total params: 4,262
Trainable params: 4,262
Non-trainable params: 0
_________________________________________________________________
Train on 3976 samples, validate on 994 samples
Epoch 1/10
 - 1s - loss: 0.6963 - acc: 0.5045 - val_loss: 0.6933 - val_acc: 0.5241
Epoch 2/10
 - 0s - loss: 0.6930 - acc: 0.5156 - val_loss: 0.6926 - val_acc: 0.5241
Epoch 3/10
 - 0s - loss: 0.6928 - acc: 0.5018 - val_loss: 0.6923 - val_acc: 0.5241
Epoch 4/10
 - 0s - loss: 0.6927 - acc: 0.5174 - val_loss: 0.6923 - val_acc: 0.5262
Epoch 5/10
 - 0s - loss: 0.6924 - acc: 0.5143 - val_loss: 0.6929 - val_acc: 0.4748
Epoch 6/10
 - 0s - loss: 0.6924 - acc: 0.5138 - val_loss: 0.6919 - val_acc

In [25]:
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

mlp = MLPClassifier(hidden_layer_sizes=(500,3))
dtc = DecisionTreeClassifier()
logReg = LogisticRegression()

In [26]:
x_train, x_test, y_train, y_test = train_test_split(dataNA[['short_sma', 'long_sma', 'previous_price']].values, dataNA['rise_or_decrease'].values, test_size=0.33, shuffle=False)
mlp.fit(x_train, y_train)
dtc.fit(x_train, y_train)
logReg.fit(x_train, y_train)
mlp_accNA = mlp.score(x_test, y_test)
dtc_accNA = dtc.score(x_test, y_test)
logReg_accNA = logReg.score(x_test, y_test)

In [27]:
x_train, x_test, y_train, y_test = train_test_split(dataCN[['short_sma', 'long_sma', 'previous_price']].values, dataCN['rise_or_decrease'].values, test_size=0.33, shuffle=False)
mlp.fit(x_train, y_train)
dtc.fit(x_train, y_train)
logReg.fit(x_train, y_train)
mlp_accCN = mlp.score(x_test, y_test)
dtc_accCN = dtc.score(x_test, y_test)
logReg_accCN = logReg.score(x_test, y_test)

In [28]:
x_train, x_test, y_train, y_test = train_test_split(dataEU[['short_sma', 'long_sma', 'previous_price']].values, dataEU['rise_or_decrease'].values, test_size=0.33, shuffle=False)
mlp.fit(x_train, y_train)
dtc.fit(x_train, y_train)
logReg.fit(x_train, y_train)
mlp_accEU = mlp.score(x_test, y_test)
dtc_accEU = dtc.score(x_test, y_test)
logReg_accEU = logReg.score(x_test, y_test)

In [29]:
x_train, x_test, y_train, y_test = train_test_split(dataKR[['short_sma', 'long_sma', 'previous_price']].values, dataKR['rise_or_decrease'].values, test_size=0.33, shuffle=False)
mlp.fit(x_train, y_train)
dtc.fit(x_train, y_train)
logReg.fit(x_train, y_train)
mlp_accKR = mlp.score(x_test, y_test)
dtc_accKR = dtc.score(x_test, y_test)
logReg_accKR = logReg.score(x_test, y_test)

In [30]:
x_train, x_test, y_train, y_test = train_test_split(dataTW[['short_sma', 'long_sma', 'previous_price']].values, dataTW['rise_or_decrease'].values, test_size=0.33, shuffle=False)
mlp.fit(x_train, y_train)
dtc.fit(x_train, y_train)
logReg.fit(x_train, y_train)
mlp_accTW = mlp.score(x_test, y_test)
dtc_accTW = dtc.score(x_test, y_test)
logReg_accTW = logReg.score(x_test, y_test)

In [31]:
print('-'*40)
print('Model\t\tRegion\t\tAccuracy')
print('-'*40)
print('LSTM\t\tNA\t\t%.6f' %lstm_accNA)
print('MLP\t\tNA\t\t%.6f' %mlp_accNA)
print('DTC\t\tNA\t\t%.6f' %dtc_accNA)
print('LogReg\t\tNA\t\t%.6f' %logReg_accNA)
print('-'*40)
print('LSTM\t\tCN\t\t%.6f' %lstm_accCN)
print('MLP\t\tCN\t\t%.6f' %mlp_accCN)
print('DTC\t\tCN\t\t%.6f' %dtc_accCN)
print('LogReg\t\tCN\t\t%.6f' %logReg_accCN)
print('-'*40)
print('LSTM\t\tEU\t\t%.6f' %lstm_accEU)
print('MLP\t\tEU\t\t%.6f' %mlp_accEU)
print('DTC\t\tEU\t\t%.6f' %dtc_accEU)
print('LogReg\t\tEU\t\t%.6f' %logReg_accEU)
print('-'*40)
print('LSTM\t\tKR\t\t%.6f' %lstm_accKR)
print('MLP\t\tKR\t\t%.6f' %mlp_accKR)
print('DTC\t\tKR\t\t%.6f' %dtc_accKR)
print('LogReg\t\tKR\t\t%.6f' %logReg_accKR)
print('-'*40)
print('LSTM\t\tTW\t\t%.6f' %lstm_accTW)
print('MLP\t\tTW\t\t%.6f' %mlp_accTW)
print('DTC\t\tTW\t\t%.6f' %dtc_accTW)
print('LogReg\t\tTW\t\t%.6f' %logReg_accTW)
print('-'*40)

----------------------------------------
Model		Region		Accuracy
----------------------------------------
LSTM		NA		0.523000
MLP		NA		0.501353
DTC		NA		0.498647
LogReg		NA		0.601082
----------------------------------------
LSTM		CN		0.505804
MLP		CN		0.517857
DTC		CN		0.530804
LogReg		CN		0.658482
----------------------------------------
LSTM		EU		0.495669
MLP		EU		0.514567
DTC		EU		0.487795
LogReg		EU		0.637402
----------------------------------------
LSTM		KR		0.545092
MLP		KR		0.511173
DTC		KR		0.545092
LogReg		KR		0.629290
----------------------------------------
LSTM		TW		0.495304
MLP		TW		0.480196
DTC		TW		0.546345
LogReg		TW		0.640261
----------------------------------------


## Conclusão

O modelo de Regressão Logística foi o que obteve maior acurácia para as previsões das tendências das moedas. Desse modo, esse modelo deve ser o mais recomendado para se realizar previsões do WoW Token, aliado ao modelo de SMA (Simple Moving Average) para melhores resultados e acurácia maior nas suas previsões.